# Activity 2 
## Extract and Repurpose Data - Part A

This notebook takes the data from the SQLite Database for European Soccer, and creates two CSV files - players.csv and teams.csv, with the required data to answer the questions.

We begin by loading the required libraries and establishing a connection the the SQLite database file (database.sqlite)

In [37]:
library(dplyr)
library(RSQLite) 
library(plyr)
library(sqldf)
library(gdata)

In [38]:
db <- dbConnect(SQLite(), dbname='database.sqlite')

In [39]:
dbListTables(db)

[1] "Country"           "League"            "Match"            
[4] "Player"            "Player_Attributes" "Team"             
[7] "Team_Attributes"   "sqlite_sequence"

Extract data from individual tables in the database and loading it into R

In [40]:
country    <- tbl_df(dbGetQuery(db,"select * from country"))
player     <- tbl_df(dbGetQuery(db,"select * from player"))
team       <- tbl_df(dbGetQuery(db,"select * from team"))
teamatt    <- tbl_df(dbGetQuery(db,"select * from team_attributes"))
playeratt  <- tbl_df(dbGetQuery(db,"select * from player_attributes"))
league     <- tbl_df(dbGetQuery(db,"select * from league"))
match      <- tbl_df(dbGetQuery(db,"select * from match"))

### Player Data Extraction and Store 

The following query is used to extract the player names, their total sum of matches played, and the number of distinct leagues the players have played in, for further manipulation.

In [41]:
query <- 'select a.player_api_id,
                 a.player_name,
                 count(a.player_api_id),
                 count(distinct league_id)
            from player a 
            inner join match b
        on a.player_api_id = b.home_player_1 
        or a.player_api_id = b.home_player_2 
        or a.player_api_id = b.home_player_3
        or a.player_api_id = b.home_player_4 
        or a.player_api_id = b.home_player_5 
        or a.player_api_id = b.home_player_6 
        or a.player_api_id = b.home_player_7 
        or a.player_api_id = b.home_player_8 
        or a.player_api_id = b.home_player_9
        or a.player_api_id = b.home_player_10
        or a.player_api_id = b.home_player_11
        or a.player_api_id = b.away_player_1
        or a.player_api_id = b.away_player_2
        or a.player_api_id = b.away_player_3
        or a.player_api_id = b.away_player_4
        or a.player_api_id = b.away_player_5 
        or a.player_api_id = b.away_player_6 
        or a.player_api_id = b.away_player_7 
        or a.player_api_id = b.away_player_8 
        or a.player_api_id = b.away_player_9
        or a.player_api_id = b.away_player_10
        or a.player_api_id = b.away_player_11 
    group by a.player_api_id'

In [42]:
players_data <- sqldf(query,stringsAsFactors = FALSE)


In [43]:
names(players_data)

[1] "player_api_id"             "player_name"              
[3] "count(a.player_api_id)"    "count(distinct league_id)"

Rename the columns in players_data with appropriate names, create dataframe topleagues having players with the number of leagues they have played in, in descending order

In [44]:
players_data <- rename.vars(players_data, c("player_api_id","player_name","count(a.player_api_id)","count(distinct league_id)"), 
                                          c("player_api_id","player_name","num_matches","num_leagues"))
topleagues<- arrange(players_data, desc (num_leagues))


Changing in players_data                                                      
From: player_api_id player_name count(a.player_api_id)
To:   player_api_id player_name num_matches           
                               
From: count(distinct league_id)
To:   num_leagues              



Extract Player Name and Player IDs. Get Player attributes, and add a column for the sum of GoalKeeper attributes.

We then keep only the max sum of goalkeeper attributes for each player, merge the data with player names, and 
arrange the data in descending order of sum of goalkeeper attributes.

In [45]:
players1 <- player %>% select(unique(player_api_id),(player_name))
playersatt1 <- playeratt %>% select(player_api_id,overall_rating, gk_diving, gk_handling, gk_kicking, gk_positioning, gk_reflexes)
playersatt1 <- mutate(playersatt1, gk_total = gk_diving+gk_handling+gk_kicking+gk_positioning+gk_reflexes)


In [46]:
q2 <- playersatt1 %>% 
# select(overall_rating,player_api_id) %>%
    group_by(player_api_id)%>%
#     distinct(player_api_id) %>%    
    top_n(n = 1, wt=gk_total) %>%
    distinct(gk_total) %>%
    as.data.frame()
q2 <- inner_join(q2,players1)
q2 = merge(player, q2, by="player_api_id")
q2 <- arrange(q2,desc(gk_total))
topgk <- select(q2,player_api_id,gk_total,player_name.x)
head(topgk,n=10)

Joining, by = "player_api_id"


player_api_id,gk_total,player_name.x
30717,449,Gianluigi Buffon
39989,447,Gregory Coupet
30859,445,Petr Cech
30657,442,Iker Casillas
27299,440,Manuel Neuer
30989,438,Julio Cesar
24503,437,Sebastian Frey
30726,436,Edwin van der Sar
182917,429,David De Gea
30660,428,Pepe Reina


Arranging player data according to max overall rating, of all time. We then merge the data with the max GK attributes, Number of Matches played by each player, and number of leagues each player has played in

In [47]:
toprated <- playersatt1 %>% 
# select(overall_rating,player_api_id) %>%
    group_by(player_api_id) %>%
#     distinct(player_api_id) %>%
#     summarise(overall_rating = max(overall_rating)) %>%
#     arrange(player_api_id,desc(overall_rating)) %>%
    top_n(n = 1, wt=overall_rating) %>%
#     top_n(n = 1, wt=player_api_id) %>%
#     distinct(overall_rating) %>%
    as.data.frame()
head(toprated)

player_api_id,overall_rating,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,gk_total
505942,67,6,11,10,8,8,43
505942,67,6,11,10,8,8,43
155782,74,14,7,9,9,12,51
155782,74,14,7,9,9,12,51
155782,74,14,7,9,9,12,51
162549,71,15,10,11,8,12,56


In [48]:
toprated<- toprated %>% distinct(player_api_id, overall_rating, .keep_all = TRUE)
toprated<- arrange(toprated,desc(overall_rating))

In [49]:
playerMerge = merge(player, toprated, by="player_api_id")
playerMerge <- arrange(playerMerge,desc(overall_rating))
play_rating <- select(playerMerge,player_api_id,player_name,overall_rating)

In [50]:
playersdata = merge(play_rating, topgk, by="player_api_id")
playersdata = select(playersdata,player_api_id,player_name,overall_rating,gk_total)
playersdata = merge(playersdata,topleagues, by="player_api_id")
playersdata = select(playersdata,player_api_id,player_name.x,overall_rating,gk_total,num_matches,num_leagues)

In [51]:
names(playersdata)

[1] "player_api_id"  "player_name.x"  "overall_rating" "gk_total"      
[5] "num_matches"    "num_leagues"

Writing the data to CSV file

In [52]:
playersdata <- rename.vars(playersdata, c("player_api_id","player_name.x"), c("player_api_id","player_name"))

write.csv(playersdata,file="players.csv",row.names=FALSE)


Changing in playersdata                                 
From: player_api_id player_name.x
To:   player_api_id player_name  



### Extract Team Data and Store 

Collect the relevant team attributes from the table and store them to teamStat. 
Set the NA attributes to 0.
Then create 3 separate dataframes to store the max BuildUp attributes, max Chance Creation attributes, and max Defence attributes for each team.


In [53]:
teamStat <- teamatt %>% select(team_api_id,buildUpPlaySpeed,
                               buildUpPlaySpeedClass,
                               buildUpPlayDribbling,
                               buildUpPlayDribblingClass,
                               buildUpPlayPassing,
                               buildUpPlayPassingClass,
                               buildUpPlayPositioningClass,
                               chanceCreationPassing,
                               chanceCreationPassingClass,
                               chanceCreationCrossing,
                               chanceCreationCrossingClass,
                               chanceCreationShooting,
                               chanceCreationShootingClass,
                               chanceCreationPositioningClass,
                               defencePressure,
                               defencePressureClass,
                               defenceAggression,
                               defenceAggressionClass,
                               defenceTeamWidth,
                               defenceTeamWidthClass,
                               defenceDefenderLineClass)

In [54]:
teamStat[is.na(teamStat)] <- 0
# Set NA values to 0, add 3 column with individual sums of BuildUp, Chance Creation and Defence Attributes
teamStat <- mutate(teamStat,bup=buildUpPlaySpeed+buildUpPlayDribbling+buildUpPlayPassing,
                   chtot=chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting,
                   deftot=defencePressure+defenceAggression+defenceTeamWidth)

Create dataframe storing the max BuildUp Attributes for each team, and removing duplicate entries.

In [55]:
teamStat1 <- teamStat %>% 
# select(overall_rating,player_api_id) %>%
    group_by(team_api_id) %>%
#     distinct(player_api_id) %>%
#     summarise(overall_rating = max(overall_rating)) %>%
#     arrange(player_api_id,desc(overall_rating)) %>%
    top_n(n = 1, wt=bup) %>%
#     top_n(n = 1, wt=player_api_id) %>%

#     distinct(overall_rating) %>%
    as.data.frame()
head(teamStat1)

team_api_id,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,...,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,bup,chtot,deftot
9930,52,Balanced,48,Normal,56,Mixed,Organised,54,Normal,...,47,Medium,44,Press,54,Normal,Cover,156,181,145
8485,58,Balanced,64,Normal,62,Mixed,Organised,56,Normal,...,41,Medium,42,Press,60,Normal,Cover,184,181,143
8576,59,Balanced,57,Normal,52,Mixed,Organised,48,Normal,...,38,Medium,47,Press,53,Normal,Cover,168,138,138
8576,59,Balanced,57,Normal,52,Mixed,Organised,48,Normal,...,38,Medium,47,Press,53,Normal,Cover,168,138,138
8564,48,Balanced,70,Lots,52,Mixed,Organised,66,Normal,...,58,Medium,57,Press,49,Normal,Cover,170,177,164
8564,48,Balanced,70,Lots,52,Mixed,Organised,66,Normal,...,58,Medium,57,Press,49,Normal,Cover,170,171,164


In [56]:
teamStat1<- teamStat1 %>% distinct(team_api_id, bup, .keep_all = TRUE)
teamStat1 <- select(teamStat1,team_api_id,bup)
teamStat1<- arrange(teamStat1,desc(bup))
head(teamStat1)

team_api_id,bup
9804,209
8600,205
8194,198
8033,193
8245,192
208931,190


Create dataframe storing the max Chance Creation Attributes for each team, and removing duplicate entries.

In [57]:
teamStat2 <- teamStat %>% 
# select(overall_rating,player_api_id) %>%
    group_by(team_api_id) %>%
#     distinct(player_api_id) %>%
#     summarise(overall_rating = max(overall_rating)) %>%
#     arrange(player_api_id,desc(overall_rating)) %>%
    top_n(n = 1, wt=chtot) %>%
#     top_n(n = 1, wt=player_api_id) %>%

#     distinct(overall_rating) %>%
    as.data.frame()
head(teamStat2)

team_api_id,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,...,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,bup,chtot,deftot
9930,52,Balanced,48,Normal,56,Mixed,Organised,54,Normal,...,47,Medium,44,Press,54,Normal,Cover,156,181,145
9930,47,Balanced,41,Normal,54,Mixed,Organised,54,Normal,...,47,Medium,44,Press,54,Normal,Cover,142,181,145
8485,70,Fast,0,Little,70,Long,Organised,70,Risky,...,60,Medium,70,Double,70,Wide,Cover,140,210,200
8576,65,Balanced,0,Little,45,Mixed,Organised,65,Normal,...,45,Medium,45,Press,50,Normal,Cover,110,180,140
8564,48,Balanced,70,Lots,52,Mixed,Organised,66,Normal,...,58,Medium,57,Press,49,Normal,Cover,170,177,164
10215,45,Balanced,0,Little,44,Mixed,Organised,55,Normal,...,52,Medium,38,Press,61,Normal,Cover,89,172,151


In [58]:
teamStat2<- teamStat2 %>% distinct(team_api_id, chtot, .keep_all = TRUE)
teamStat2 <- select(teamStat2,team_api_id,chtot)
teamStat2<- arrange(teamStat2,desc(chtot))
head(teamStat2)

team_api_id,chtot
8543,220
8697,216
9875,215
8485,210
8178,210
8658,210


Create dataframe storing the max Defence Attributes for each team, and removing duplicate entries.

In [59]:
teamStat3 <- teamStat %>% 
# select(overall_rating,player_api_id) %>%
    group_by(team_api_id) %>%
#     distinct(player_api_id) %>%
#     summarise(overall_rating = max(overall_rating)) %>%
#     arrange(player_api_id,desc(overall_rating)) %>%
    top_n(n = 1, wt=deftot) %>%
#     top_n(n = 1, wt=player_api_id) %>%

#     distinct(overall_rating) %>%
    as.data.frame()
head(teamStat3)

team_api_id,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,...,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass,bup,chtot,deftot
9930,60,Balanced,0,Little,50,Mixed,Organised,60,Normal,...,50,Medium,55,Press,45,Normal,Cover,110,180,150
8485,70,Fast,0,Little,70,Long,Organised,70,Risky,...,60,Medium,70,Double,70,Wide,Cover,140,210,200
8576,65,Balanced,0,Little,45,Mixed,Organised,65,Normal,...,45,Medium,45,Press,50,Normal,Cover,110,180,140
8564,48,Balanced,70,Lots,52,Mixed,Organised,66,Normal,...,58,Medium,57,Press,49,Normal,Cover,170,177,164
8564,48,Balanced,70,Lots,52,Mixed,Organised,66,Normal,...,58,Medium,57,Press,49,Normal,Cover,170,171,164
10215,45,Balanced,0,Little,44,Mixed,Organised,55,Normal,...,52,Medium,38,Press,61,Normal,Cover,89,172,151


In [60]:
teamStat3<- teamStat3 %>% distinct(team_api_id, deftot, .keep_all = TRUE)
teamStat3 <- select(teamStat3,team_api_id,deftot)
teamStat3<- arrange(teamStat3,desc(deftot))
head(teamStat3)

team_api_id,deftot
9993,210
8658,210
8483,210
8678,210
8521,210
8344,210


Merging the dataframes, each containing max of the sum of the BuildUp, Chance Creation and Defence Attributes, 
and renaming the resultant column names

In [61]:
teamComb <- merge(team,teamStat1)
teamComb <- merge(teamComb,teamStat2)
teamComb <- merge(teamComb,teamStat3)
head(teamComb)
teamComb<- select(teamComb,team_api_id,team_long_name,bup,chtot,deftot)
teamComb <- rename.vars(teamComb, c("team_api_id","team_long_name","bup","chtot","deftot"), 
                                  c("team_api_id","team_name","build_up","chance_cr","defence"))


team_api_id,id,team_fifa_api_id,team_long_name,team_short_name,bup,chtot,deftot
1601,31446,874,Ruch Chorzów,CHO,139,186,175
1773,1513,100087,Oud-Heverlee Leuven,O-H,153,157,137
1957,31456,110745,Jagiellonia Bialystok,BIA,158,163,210
2033,35774,111540,S.C. Olhanense,OLH,147,165,155
2182,31453,873,Lech Poznan,POZ,176,196,160
2183,31448,1570,P. Warszawa,PWA,80,180,175



Changing in teamComb                                                           
From: team_api_id team_long_name bup      chtot     deftot 
To:   team_api_id team_name      build_up chance_cr defence



Find number of home and away goals by each team, 
adding a column for total goals by each team 
and merging them with the above dataframe

In [62]:
#Find number of home goals by each team
query1 <- 'select a.team_api_id,
                  a.team_long_name,
                 sum(b.home_team_goal)
            from  team a 
            inner join match b
        on a.team_api_id = b.home_team_api_id 
        group by a.team_api_id'
home_goals <- sqldf(query1,stringsAsFactors = FALSE)


In [63]:
#Find number of away goals by each team
query2 <- 'select a.team_api_id,
                 sum(b.away_team_goal)
            from  team a 
            inner join match b
        on a.team_api_id = b.away_team_api_id 
        group by a.team_api_id'
away_goals <- sqldf(query2,stringsAsFactors = FALSE)


In [64]:
#Merge both, rename and add total_goals
teamGoals <- merge(home_goals,away_goals,by="team_api_id")
teamGoals <- rename.vars(teamGoals, c("team_api_id","team_long_name","sum(b.home_team_goal)","sum(b.away_team_goal)"), 
                                    c("team_api_id","team_name","home_goals","away_goals"))

teamGoals<- mutate(teamGoals,total_goals=home_goals+away_goals)



Changing in teamGoals                                                                            
From: team_api_id team_long_name sum(b.home_team_goal) sum(b.away_team_goal)
To:   team_api_id team_name      home_goals            away_goals           



Merge the goal data and team data collected before, and write the CSV file

In [65]:
#Merge with Team Data collected before
teamComb<-merge(teamComb,teamGoals)
#Write CSV
write.csv(teamComb,"teams.csv",row.names=FALSE)